# Mixed Binary Optimization applied to Transaction Settlement
### Reference: https://arxiv.org/abs/1910.05788

In [1]:
# Importing the libraries
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import InequalityToEquality, LinearEqualityToPenalty
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit import IBMQ
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import COBYLA
from qiskit.opflow import CVaRExpectation

In [2]:
# Creating a Quadratic Program for the Qubit-based Transaction Settlement Problem
# Test case 1:
test_case_1 = QuadraticProgram("test_case_1")
print(test_case_1.prettyprint())

Problem name: test_case_1

Minimize
  0

Subject to
  No constraints

  No variables



In [3]:
# Adding the variables to the Quadratic Program
test_case_1.binary_var("x1")
test_case_1.binary_var("x2")
test_case_1.binary_var("x3")
print(test_case_1.prettyprint())

Problem name: test_case_1

Minimize
  0

Subject to
  No constraints

  Binary variables (3)
    x1 x2 x3



In [4]:
# Adding the objective function to the Quadratic Program
test_case_1.maximize(linear=[1, 1, 1])
print(test_case_1.prettyprint())

Problem name: test_case_1

Maximize
  x1 + x2 + x3

Subject to
  No constraints

  Binary variables (3)
    x1 x2 x3



In [5]:
# Adding the constraints to the Quadratic Program
test_case_1.linear_constraint(linear=[-1, 0, -1], sense=">=", rhs=-2, name="p1_c")
test_case_1.linear_constraint(linear=[2, 0, 2], sense=">=", rhs=0, name="p1_s")
test_case_1.linear_constraint(linear=[1, 1, 0], sense=">=", rhs=0, name="p2_c")
test_case_1.linear_constraint(linear=[-2, -2, 0], sense=">=", rhs=-3, name="p2_s")
test_case_1.linear_constraint(linear=[0, -1, 1], sense=">=", rhs=0, name="p3_c")
test_case_1.linear_constraint(linear=[0, 2, -2], sense=">=", rhs=0, name="p3_s")
print(test_case_1.prettyprint())

Problem name: test_case_1

Maximize
  x1 + x2 + x3

Subject to
  Linear constraints (6)
    -x1 - x3 >= -2  'p1_c'
    2*x1 + 2*x3 >= 0  'p1_s'
    x1 + x2 >= 0  'p2_c'
    -2*x1 - 2*x2 >= -3  'p2_s'
    -x2 + x3 >= 0  'p3_c'
    2*x2 - 2*x3 >= 0  'p3_s'

  Binary variables (3)
    x1 x2 x3



In [6]:
# Converting the Quadratic Program to an Equality Constraint Program
ineq2eq = InequalityToEquality()
test_case_1_eq = ineq2eq.convert(test_case_1)
print(test_case_1_eq.prettyprint())

Problem name: test_case_1

Maximize
  x1 + x2 + x3

Subject to
  Linear constraints (6)
    -p1_c@int_slack - x1 - x3 == -2  'p1_c'
    -p1_s@int_slack + 2*x1 + 2*x3 == 0  'p1_s'
    -p2_c@int_slack + x1 + x2 == 0  'p2_c'
    -p2_s@int_slack - 2*x1 - 2*x2 == -3  'p2_s'
    -p3_c@int_slack - x2 + x3 == 0  'p3_c'
    -p3_s@int_slack + 2*x2 - 2*x3 == 0  'p3_s'

  Integer variables (6)
    0 <= p1_c@int_slack <= 2
    0 <= p1_s@int_slack <= 4
    0 <= p2_c@int_slack <= 2
    0 <= p2_s@int_slack <= 3
    0 <= p3_c@int_slack <= 1
    0 <= p3_s@int_slack <= 2

  Binary variables (3)
    x1 x2 x3



In [7]:
# Converting Equality Constraints to Penalty Functions
linear_eq2penalty = LinearEqualityToPenalty(penalty=1e3)
qubo = linear_eq2penalty.convert(test_case_1_eq)
print(qubo.prettyprint())

Problem name: test_case_1

Maximize
  -1000*p1_c@int_slack^2 - 1000*p1_s@int_slack^2 - 1000*p2_c@int_slack^2
  - 1000*p2_s@int_slack^2 - 1000*p3_c@int_slack^2 - 1000*p3_s@int_slack^2
  - 2000*x1*p1_c@int_slack + 4000*x1*p1_s@int_slack + 2000*x1*p2_c@int_slack
  - 4000*x1*p2_s@int_slack - 10000*x1^2 - 10000*x1*x2 - 10000*x1*x3
  + 2000*x2*p2_c@int_slack - 4000*x2*p2_s@int_slack - 2000*x2*p3_c@int_slack
  + 4000*x2*p3_s@int_slack - 10000*x2^2 + 10000*x2*x3 - 2000*x3*p1_c@int_slack
  + 4000*x3*p1_s@int_slack + 2000*x3*p3_c@int_slack - 4000*x3*p3_s@int_slack
  - 10000*x3^2 + 4000*p1_c@int_slack + 6000*p2_s@int_slack + 16001*x1 + 12001*x2
  + 4001*x3 - 13000

Subject to
  No constraints

  Integer variables (6)
    0 <= p1_c@int_slack <= 2
    0 <= p1_s@int_slack <= 4
    0 <= p2_c@int_slack <= 2
    0 <= p2_s@int_slack <= 3
    0 <= p3_c@int_slack <= 1
    0 <= p3_s@int_slack <= 2

  Binary variables (3)
    x1 x2 x3



In [10]:
# Loading the IBMQ Backend
provider = IBMQ.enable_account("your-token-here")

In [11]:
backend = provider.get_backend("simulator_statevector")
# backend = provider.get_backend("ibmq_qasm_simulator")
print(backend.name())

simulator_statevector


In [12]:
# Creating a Quantum Instance
algorithm_globals.random_seed = 1234
quantum_instance = QuantumInstance(backend,
                                   shots=8192,
                                   seed_simulator=algorithm_globals.random_seed,
                                   seed_transpiler=algorithm_globals.random_seed)

In [13]:
# Initializing the MinimumEigenSolver
qaoa_mes = QAOA(optimizer=COBYLA(maxiter=150), expectation=CVaRExpectation(alpha=0.25),
                quantum_instance=quantum_instance)
exact_mes = NumPyMinimumEigensolver()

In [14]:
# Creating the MinimumEigenOptimizer for the MinimumEigenSolver
qaoa = MinimumEigenOptimizer(qaoa_mes)
exact = MinimumEigenOptimizer(exact_mes)

In [ ]:
exact_result = exact.solve(qubo)
print(exact_result.prettyprint())

objective function value: 2.0
variable values: x1=0.0, x2=1.0, x3=1.0, p1_c@int_slack=1.0, p1_s@int_slack=2.0, p2_c@int_slack=1.0, p2_s@int_slack=1.0, p3_c@int_slack=0.0, p3_s@int_slack=0.0
status: SUCCESS


In [ ]:
# Solving the Quadratic Program using the MinimumEigenOptimizer
qaoa_result = qaoa.solve(qubo)
print(qaoa_result.prettyprint())

objective function value: 2.0
variable values: x1=0.0, x2=1.0, x3=1.0, p1_c@int_slack=1.0, p1_s@int_slack=2.0, p2_c@int_slack=1.0, p2_s@int_slack=1.0, p3_c@int_slack=0.0, p3_s@int_slack=0.0
status: SUCCESS


In [15]:
# Creating a Quadratic Program for the Qubit-based Transaction Settlement Problem
# Test case 2:
test_case_2 = QuadraticProgram("test_case_2")
print(test_case_2.prettyprint())

Problem name: test_case_2

Minimize
  0

Subject to
  No constraints

  No variables



In [16]:
# Adding the variables to the Quadratic Program
test_case_2.binary_var_list(["1", "2", "3"])
# upper bounds calculated by reading the paper
test_case_2.integer_var(name="lc", lowerbound=-2, upperbound=0)
test_case_2.integer_var(name="ls", lowerbound=0, upperbound=1)
print(test_case_2.prettyprint())

Problem name: test_case_2

Minimize
  0

Subject to
  No constraints

  Integer variables (2)
    -2 <= lc <= 0
    0 <= ls <= 1

  Binary variables (3)
    x1 x2 x3



In [17]:
# Adding the objective function to the Quadratic Program
test_case_2.maximize(linear=[1] * 3)
print(test_case_2.prettyprint())

Problem name: test_case_2

Maximize
  x1 + x2 + x3

Subject to
  No constraints

  Integer variables (2)
    -2 <= lc <= 0
    0 <= ls <= 1

  Binary variables (3)
    x1 x2 x3



In [18]:
# Adding the constraints to the Quadratic Program
test_case_2.linear_constraint(linear={"x1": -1, "x3": -1, "lc": -1}, sense=">=", rhs=0, name="p1_c")
test_case_2.linear_constraint(linear={"x1": 2, "x3": 2, "ls": -1}, sense=">=", rhs=-1, name="p1_s")
test_case_2.linear_constraint(linear={"lc": 1, "ls": 2}, sense="==", rhs=0, name="f1")
test_case_2.linear_constraint(linear=[1, 1], sense=">=", rhs=0, name="p2_c")
test_case_2.linear_constraint(linear=[-2, -2], sense=">=", rhs=-3, name="p2_s")
test_case_2.linear_constraint(linear=[0, -1, 1], sense=">=", rhs=0, name="p3_c")
test_case_2.linear_constraint(linear=[0, 2, -2], sense=">=", rhs=0, name="p3_s")
print(test_case_2.prettyprint())

Problem name: test_case_2

Maximize
  x1 + x2 + x3

Subject to
  Linear constraints (7)
    -lc - x1 - x3 >= 0  'p1_c'
    -ls + 2*x1 + 2*x3 >= -1  'p1_s'
    lc + 2*ls == 0  'f1'
    x1 + x2 >= 0  'p2_c'
    -2*x1 - 2*x2 >= -3  'p2_s'
    -x2 + x3 >= 0  'p3_c'
    2*x2 - 2*x3 >= 0  'p3_s'

  Integer variables (2)
    -2 <= lc <= 0
    0 <= ls <= 1

  Binary variables (3)
    x1 x2 x3



In [19]:
# Converting the Quadratic Program to an Equality Constraint Program
ineq2eq = InequalityToEquality()
test_case_2_eq = ineq2eq.convert(test_case_2)
print(test_case_2_eq.prettyprint())

Problem name: test_case_2

Maximize
  x1 + x2 + x3

Subject to
  Linear constraints (7)
    -lc - p1_c@int_slack - x1 - x3 == 0  'p1_c'
    -ls - p1_s@int_slack + 2*x1 + 2*x3 == -1  'p1_s'
    lc + 2*ls == 0  'f1'
    -p2_c@int_slack + x1 + x2 == 0  'p2_c'
    -p2_s@int_slack - 2*x1 - 2*x2 == -3  'p2_s'
    -p3_c@int_slack - x2 + x3 == 0  'p3_c'
    -p3_s@int_slack + 2*x2 - 2*x3 == 0  'p3_s'

  Integer variables (8)
    -2 <= lc <= 0
    0 <= ls <= 1
    0 <= p1_c@int_slack <= 2
    0 <= p1_s@int_slack <= 5
    0 <= p2_c@int_slack <= 2
    0 <= p2_s@int_slack <= 3
    0 <= p3_c@int_slack <= 1
    0 <= p3_s@int_slack <= 2

  Binary variables (3)
    x1 x2 x3



In [20]:
# Converting Equality Constraints to Penalty Functions
linear_eq2penalty = LinearEqualityToPenalty(penalty=1e3)
qubo_2 = linear_eq2penalty.convert(test_case_2_eq)
print(qubo_2.prettyprint())

Problem name: test_case_2

Maximize
  -2000*lc^2 - 4000*lc*ls - 2000*lc*p1_c@int_slack - 5000*ls^2
  - 2000*ls*p1_s@int_slack - 1000*p1_c@int_slack^2 - 1000*p1_s@int_slack^2
  - 1000*p2_c@int_slack^2 - 1000*p2_s@int_slack^2 - 1000*p3_c@int_slack^2
  - 1000*p3_s@int_slack^2 - 2000*x1*lc + 4000*x1*ls - 2000*x1*p1_c@int_slack
  + 4000*x1*p1_s@int_slack + 2000*x1*p2_c@int_slack - 4000*x1*p2_s@int_slack
  - 10000*x1^2 - 10000*x1*x2 - 10000*x1*x3 + 2000*x2*p2_c@int_slack
  - 4000*x2*p2_s@int_slack - 2000*x2*p3_c@int_slack + 4000*x2*p3_s@int_slack
  - 10000*x2^2 + 10000*x2*x3 - 2000*x3*lc + 4000*x3*ls - 2000*x3*p1_c@int_slack
  + 4000*x3*p1_s@int_slack + 2000*x3*p3_c@int_slack - 4000*x3*p3_s@int_slack
  - 10000*x3^2 + 2000*ls + 2000*p1_s@int_slack + 6000*p2_s@int_slack + 8001*x1
  + 12001*x2 - 3999*x3 - 10000

Subject to
  No constraints

  Integer variables (8)
    -2 <= lc <= 0
    0 <= ls <= 1
    0 <= p1_c@int_slack <= 2
    0 <= p1_s@int_slack <= 5
    0 <= p2_c@int_slack <= 2
    0 <= p

In [21]:
algorithm_globals.massive = True
exact_result_2 = exact.solve(qubo_2)
print(exact_result_2.prettyprint())

objective function value: 2.0
variable values: x1=0.0, x2=1.0, x3=1.0, lc=-2.0, ls=1.0, p1_c@int_slack=1.0, p1_s@int_slack=2.0, p2_c@int_slack=1.0, p2_s@int_slack=1.0, p3_c@int_slack=0.0, p3_s@int_slack=0.0
status: SUCCESS


In [22]:
# Solving the Quadratic Program using the MinimumEigenOptimizer
qaoa_result_2 = qaoa.solve(qubo_2)
print(qaoa_result_2.prettyprint())

objective function value: 2.0
variable values: x1=0.0, x2=1.0, x3=1.0, lc=-2.0, ls=1.0, p1_c@int_slack=1.0, p1_s@int_slack=2.0, p2_c@int_slack=1.0, p2_s@int_slack=1.0, p3_c@int_slack=0.0, p3_s@int_slack=0.0
status: SUCCESS


In [23]:
# Creating a Quadratic Program for the Qubit-based Transaction Settlement Problem
# Test case 3:
test_case_3 = QuadraticProgram("test_case_3")
print(test_case_3.prettyprint())

Problem name: test_case_3

Minimize
  0

Subject to
  No constraints

  No variables



In [24]:
# Adding the variables to the Quadratic Program
test_case_3.binary_var_list(["1", "2", "3", "4", "5", "6", "7"])
print(test_case_3.prettyprint())

Problem name: test_case_3

Minimize
  0

Subject to
  No constraints

  Binary variables (7)
    x1 x2 x3 x4 x5 x6 x7



In [25]:
# Adding the objective function to the Quadratic Program
test_case_3.maximize(linear=[1] * 7)
print(test_case_3.prettyprint())

Problem name: test_case_3

Maximize
  x1 + x2 + x3 + x4 + x5 + x6 + x7

Subject to
  No constraints

  Binary variables (7)
    x1 x2 x3 x4 x5 x6 x7



In [26]:
# Adding the constraints to the Quadratic Program
# Ignored P3 and P4 as mentioned in the paper
test_case_3.linear_constraint(linear=[-4, -3, -2, 0, 0, 0, 0], sense=">=", rhs=-5, name="p1")
test_case_3.linear_constraint(linear=[4, 0, 0, -3, -3, 0, 0], sense=">=", rhs=-1, name="p2")
test_case_3.linear_constraint(linear={"x4": 3, "x6": -6, "x7": 4}, sense=">=", rhs=-2, name="p5")
test_case_3.linear_constraint(linear=[0, 0, 0, 0, 3, 6, -4], sense=">=", rhs=-1, name="p6")
print(test_case_3.prettyprint())

Problem name: test_case_3

Maximize
  x1 + x2 + x3 + x4 + x5 + x6 + x7

Subject to
  Linear constraints (4)
    -4*x1 - 3*x2 - 2*x3 >= -5  'p1'
    4*x1 - 3*x4 - 3*x5 >= -1  'p2'
    3*x4 - 6*x6 + 4*x7 >= -2  'p5'
    3*x5 + 6*x6 - 4*x7 >= -1  'p6'

  Binary variables (7)
    x1 x2 x3 x4 x5 x6 x7



In [27]:
# Converting the Quadratic Program to an Equality Constraint Program
ineq2eq = InequalityToEquality()
test_case_3_eq = ineq2eq.convert(test_case_3)
print(test_case_3_eq.prettyprint())

Problem name: test_case_3

Maximize
  x1 + x2 + x3 + x4 + x5 + x6 + x7

Subject to
  Linear constraints (4)
    -p1@int_slack - 4*x1 - 3*x2 - 2*x3 == -5  'p1'
    -p2@int_slack + 4*x1 - 3*x4 - 3*x5 == -1  'p2'
    -p5@int_slack + 3*x4 - 6*x6 + 4*x7 == -2  'p5'
    -p6@int_slack + 3*x5 + 6*x6 - 4*x7 == -1  'p6'

  Integer variables (4)
    0 <= p1@int_slack <= 5
    0 <= p2@int_slack <= 5
    0 <= p5@int_slack <= 9
    0 <= p6@int_slack <= 10

  Binary variables (7)
    x1 x2 x3 x4 x5 x6 x7



In [28]:
# Converting Equality Constraints to Penalty Functions
linear_eq2penalty = LinearEqualityToPenalty(penalty=1e3)
qubo_3 = linear_eq2penalty.convert(test_case_3_eq)
print(qubo_3.prettyprint())

Problem name: test_case_3

Maximize
  -1000*p1@int_slack^2 - 1000*p2@int_slack^2 - 1000*p5@int_slack^2
  - 1000*p6@int_slack^2 - 8000*x1*p1@int_slack + 8000*x1*p2@int_slack
  - 32000*x1^2 - 24000*x1*x2 - 16000*x1*x3 + 24000*x1*x4 + 24000*x1*x5
  - 6000*x2*p1@int_slack - 9000*x2^2 - 12000*x2*x3 - 4000*x3*p1@int_slack
  - 4000*x3^2 - 6000*x4*p2@int_slack + 6000*x4*p5@int_slack - 18000*x4^2
  - 18000*x4*x5 + 36000*x4*x6 - 24000*x4*x7 - 6000*x5*p2@int_slack
  + 6000*x5*p6@int_slack - 18000*x5^2 - 36000*x5*x6 + 24000*x5*x7
  - 12000*x6*p5@int_slack + 12000*x6*p6@int_slack - 72000*x6^2 + 96000*x6*x7
  + 8000*x7*p5@int_slack - 8000*x7*p6@int_slack - 32000*x7^2
  + 10000*p1@int_slack + 2000*p2@int_slack + 4000*p5@int_slack
  + 2000*p6@int_slack + 32001*x1 + 30001*x2 + 20001*x3 - 5999*x4 + x5 + 12001*x6
  - 7999*x7 - 31000

Subject to
  No constraints

  Integer variables (4)
    0 <= p1@int_slack <= 5
    0 <= p2@int_slack <= 5
    0 <= p5@int_slack <= 9
    0 <= p6@int_slack <= 10

  Binary v

In [29]:
# Initializing the MinimumEigenSolver
qaoa_mes = QAOA(optimizer=COBYLA(maxiter=300), expectation=CVaRExpectation(alpha=0.25),
                quantum_instance=quantum_instance)
exact_mes = NumPyMinimumEigensolver()

In [30]:
# Creating the MinimumEigenOptimizer for the MinimumEigenSolver
qaoa = MinimumEigenOptimizer(qaoa_mes)
exact = MinimumEigenOptimizer(exact_mes)

In [ ]:
algorithm_globals.massive = True
exact_result_3 = exact.solve(qubo_3)
print(exact_result_3.prettyprint())

objective function value: 4.0
variable values: x1=1.0, x2=0.0, x3=0.0, x4=0.0, x5=1.0, x6=1.0, x7=1.0, p1@int_slack=1.0, p2@int_slack=2.0, p5@int_slack=0.0, p6@int_slack=6.0
status: SUCCESS


In [31]:
# Solving the Quadratic Program using the MinimumEigenOptimizer
qaoa_result_3 = qaoa.solve(qubo_3)
print(qaoa_result_3.prettyprint())

objective function value: -996.0
variable values: x1=0.0, x2=1.0, x3=1.0, x4=0.0, x5=0.0, x6=1.0, x7=1.0, p1@int_slack=0.0, p2@int_slack=1.0, p5@int_slack=0.0, p6@int_slack=4.0
status: SUCCESS


In [1]:
import qiskit.tools.jupyter
%qiskit_version_table

Version Information Qiskit Software Version qiskit-terra 0.21.0 qiskit-aer 0.10.4 qiskit-ibmq-provider 0.19.2 qiskit 0.37.0 qiskit-optimization 0.4.0 System information Python version 3.8.12 Python compiler GCC 11.2.0 Python build default, Oct 9 2022 15:59:00 OS Linux CPUs 2 Memory (Gb) 3.8291244506835938 Thu Oct 13 03:42:28 2022 UTC